In [1]:
import sys
sys.path.append('scripts')

import pandas as pd
import warnings

# Suppress all UserWarnings
warnings.filterwarnings("ignore", category=UserWarning)


In [2]:
%load_ext autoreload
%autoreload 2
from utils import *
from key_metrics import *

Database found at: c:\Users\kenia\OneDrive\Documents\GitHub\cobre\scripts\..\data\olist.sqlite


In [47]:
%reload_ext autoreload

## Load data

At the moment of loading data automatically is generated an express and basic EDA analysis

In [4]:
orders_df = load_table('orders')
order_payments_df = load_table('order_payments')
customers_df = load_table('customers')
products_df = load_table('products')
order_items_df = load_table('order_items')
order_reviews_df = load_table('order_reviews')
product_category_name_translation_df = load_table('product_category_name_translation')
sellers_df = load_table('sellers')
geolocation_df = load_table('geolocation')
leads_qualified_df = load_table('leads_qualified')
leads_closed_df = load_table('leads_closed')
# customer = customer_view()

In [5]:
# basic preprocessing
orders_df['order_purchase_timestamp'] = pd.to_datetime(orders_df['order_purchase_timestamp'])
orders_df['order_approved_at'] = pd.to_datetime(orders_df['order_approved_at'])
orders_df['order_delivered_carrier_date'] = pd.to_datetime(orders_df['order_delivered_carrier_date'])
orders_df['order_delivered_customer_date'] = pd.to_datetime(orders_df['order_delivered_customer_date'])
orders_df['order_estimated_delivery_date'] = pd.to_datetime(orders_df['order_estimated_delivery_date'])

order_items_df['shipping_limit_date'] = pd.to_datetime(order_items_df['shipping_limit_date'])

order_reviews_df['review_creation_date'] = pd.to_datetime(order_reviews_df['review_creation_date'])
order_reviews_df['review_answer_timestamp'] = pd.to_datetime(order_reviews_df['review_answer_timestamp'])

leads_qualified_df['first_contact_date'] = pd.to_datetime(leads_qualified_df['first_contact_date'])
leads_closed_df['won_date'] = pd.to_datetime(leads_closed_df['won_date'])

In [27]:
pyments_categorical_cols = explore_dataframe(order_payments_df, 'order_payments', detailed=True)
orders_categorical_cols = explore_dataframe(orders_df, 'orders', detailed=True)
order_items_categorical_cols = explore_dataframe(order_items_df, 'order_items', detailed=True)
order_reviews_categorical_cols= explore_dataframe(order_reviews_df, 'order_reviews', detailed=True)
products_categorical_cols = explore_dataframe(products_df, 'products', detailed=True)
product_category_name_translation_categorical_cols = explore_dataframe(product_category_name_translation_df, 'product_category_name_translation', detailed=True)
sellers_categorical_cols = explore_dataframe(sellers_df, 'sellers', detailed=True)
geolocalization_categorical_cols = explore_dataframe(geolocation_df, 'geolocation', detailed=True)
leads_qualified_categorical_cols = explore_dataframe(leads_qualified_df, 'leads_qualified', detailed=True)
leads_closed_categorical_cols = explore_dataframe(leads_closed_df, 'leads_closed', detailed=True)


Database found at: c:\Users\kenia\OneDrive\Documents\GitHub\cobre\scripts\..\data\olist.sqlite

DATAFRAME EXPLORATION REPORT - ORDER_PAYMENTS

Dataset shape: 103886 rows × 5 columns
Memory usage: 17.81 MB

────────────────────────────────────────────────────────────
DETAILED COLUMN ANALYSIS
────────────────────────────────────────────────────────────

DATAFRAME STRUCTURE ANALYSIS

Total columns: 5
Total rows: 103886

Column type distribution:
likely_type
Numeric        3
ID             1
Categorical    1
Name: count, dtype: int64

Column Details:
            column_name data_type  null_count null_percentage  unique_count unique_percentage                                                                                           sample_values  likely_type  min       max        mean  median
0              order_id    object           0            0.0%         99440             95.7%  [b81ef226f3fe1789b1e8b2acac839d17, a9810da82917af2d9aefd1278f1dcfa0, 25e8ea4e93396b6fa0d3dd708e76c1bd]    

## Key Metrics and Visualization

In [8]:
conversion_rate_by_payment_method(orders_df, order_payments_df)

,payment_type,conversion_rate,total_payment_value
1,credit_card,0.769351,12542084.19
0,boleto,0.198952,2869361.27
4,voucher,0.038877,379436.87
2,debit_card,0.015366,217989.79
3,not_defined,0.000030,0.00


In [11]:
effectiveness_rate, promotions_by_seller, promotions_by_category = promotions_effectiveness_analysis(orders_df, order_payments_df, order_items_df, sellers_df, products_df, product_category_name_translation_df)
print(f"Overall Effectiveness Rate of Promotions: {effectiveness_rate:.2%}")
display(promotions_by_seller.head(), promotions_by_category.head())

Overall Effectiveness Rate of Promotions: 3.89%


,seller_id,promo_orders_count,total_orders,promo_effectiveness_rate
1017,fa18c782a4866d313cadab567097f8f1,1,1,1.0
1027,fc0b214b59a83615fea981c6424a02ae,1,1,1.0
411,6576fd3e23c88f0e5d4d23f39bba0542,1,1,1.0
175,28d162ab86bb49d719c32f0ee6186eaa,1,1,1.0
139,1fe61aa5494bd3ae92a2c13e067c810f,1,1,1.0


,product_category_name_english,promo_orders_count,total_orders,promo_effectiveness_rate
47,la_cuisine,2,13,0.153846
26,fashion_male_clothing,10,112,0.089286
41,home_comfort_2,2,24,0.083333
25,fashion_bags_accessories,132,1864,0.070815
29,fashion_underwear_beach,8,121,0.066116


In [12]:
order_reviews_df.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,None,None,2018-01-18,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,None,None,2018-03-10,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,None,None,2018-02-17,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,None,Recebi bem antes do prazo estipulado.,2017-04-21,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,None,Parabéns lojas lannister adorei comprar pela I...,2018-03-01,2018-03-02 10:26:53


In [13]:
summary_df, metrics_dict, time_series_df = order_status_analysis(orders_df, order_items_df, customers_df, products_df)
summary_df

,order_status,total_orders,unique_customers,percentage
2,created,5,5,0.01
0,approved,2,2,0.00
4,invoiced,314,314,0.32
5,processing,301,301,0.30
6,shipped,1107,1107,1.11
3,delivered,96478,96478,97.02
7,unavailable,609,609,0.61
1,canceled,625,625,0.63


In [28]:
metrics_dict

Database found at: c:\Users\kenia\OneDrive\Documents\GitHub\cobre\scripts\..\data\olist.sqlite


{'total_orders': 99441,
 'delivered_rate': 97.02,
 'canceled_rate': 0.63,
 'in_transit': 1408,
 'pending_approval': 5}

In [29]:
time_series_df

order_status,date,approved,canceled,created,delivered,invoiced,processing,shipped,unavailable
0,2016-09-04,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,2016-09-05,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2016-09-13,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2016-09-15,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,2016-10-02,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
629,2018-09-29,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
630,2018-10-01,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
631,2018-10-03,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
632,2018-10-16,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
funnel_df, dict_funnel = order_fulfillment_funnel(orders_df)
print(dict_funnel)
funnel_df

{'canceled_count': 625, 'canceled_rate': 0.6285133898492574}


,stage,count,percentage
0,Created,5,0.005028
1,Approved,2,0.002011
2,Invoiced,314,0.315765
3,Shipped,1107,1.113223
4,Delivered,96478,97.020344


In [32]:
delivery_performance_by_state(orders_df, customers_df)

,state,total_orders,delivery_rate,cancellation_rate,in_transit
0,AC,81.0,98.77,0.00,1.0
3,AP,68.0,98.53,0.00,1.0
7,ES,2033.0,98.13,0.44,19.0
11,MS,715.0,98.04,0.28,6.0
2,AM,148.0,97.97,0.00,2.0
26,TO,280.0,97.86,0.36,4.0
22,RS,5466.0,97.79,0.46,51.0
19,RN,485.0,97.73,0.00,8.0
12,MT,907.0,97.68,0.22,15.0
17,PR,5045.0,97.58,0.44,44.0


In [33]:
rfm, segment_summary, recommendations = rfm_customer_segmentation(orders_df, order_payments_df)


In [34]:
rfm

,customer_id,recency,frequency,monetary,r_quartile,f_quartile,m_quartile,rfm_score,segment
0,00012a2ce6f8dcda20d059ce98491703,337,1,114.74,2,1,3,213,Hibernating
1,000161a058600d5901f007fab4c27140,458,1,67.41,1,1,2,112,Hibernating
2,0001fd6190edaaf884bcaf3d49edf079,596,1,195.42,1,1,4,114,Hibernating
3,0002414f95344307404f0ace7a26f1d5,427,1,179.35,1,1,4,114,Hibernating
4,000379cdec625522490c315e70c7a9fb,198,1,107.01,3,1,3,313,Regular
...,...,...,...,...,...,...,...,...,...
99436,fffecc9f79fd8c764f843e9951b11341,202,1,81.36,3,4,2,342,Potential Loyalists
99437,fffeda5b6d849fbd39689bb92087f431,148,1,63.13,4,4,2,442,Potential Loyalists
99438,ffff42319e9b2d713724ae527742af25,126,1,214.13,4,4,4,444,Champions
99439,ffffa3172527f765de70084a7e53aae8,410,1,45.50,1,4,1,141,Price Sensitive


In [35]:
segment_summary

,segment,customer_count,avg_recency_days,avg_frequency,avg_monetary,percentage
0,Champions,1623,108.08,1.0,409.26,1.63
2,Loyal Customers,23346,299.76,1.0,248.39,23.48
4,Potential Loyalists,24590,162.77,1.0,111.65,24.73
3,New Customers,6348,108.30,1.0,167.61,6.38
6,Regular,3021,217.94,1.0,255.60,3.04
5,Price Sensitive,15722,379.57,1.0,62.43,15.81
1,Hibernating,24791,417.02,1.0,160.65,24.93


In [36]:
recommendations

{'Champions': {'strategy': 'Reward & Retain',
  'actions': ['VIP early access to new products',
   'Exclusive discounts',
   'Personal account manager'],
  'discount_range': '10-15%',
  'communication': 'Premium channels, personalized content'},
 'Loyal Customers': {'strategy': 'Upsell & Appreciate',
  'actions': ['Loyalty program enrollment',
   'Bundle offers',
   'Referral incentives'],
  'discount_range': '10-20%',
  'communication': 'Regular engagement, product recommendations'},
 'Potential Loyalists': {'strategy': 'Engage & Convert',
  'actions': ['Membership offers',
   'Product recommendations',
   'Onboarding support'],
  'discount_range': '15-25%',
  'communication': 'Educational content, success stories'},
 'New Customers': {'strategy': 'Onboard & Nurture',
  'actions': ['Welcome series',
   'First-purchase discount',
   'Product education'],
  'discount_range': '20-30%',
  'communication': 'Onboarding emails, getting started guides'},
 'At Risk': {'strategy': 'Re-engage Ur

In [38]:
customer_lifetime_value(orders_df, order_payments_df, months_ahead=6)

,customer_id,first_purchase,last_purchase,purchase_count,total_value,avg_order_value,customer_age_days,customer_age_months,purchase_frequency,predicted_purchases,predicted_clv,clv_tier
0,00012a2ce6f8dcda20d059ce98491703,2017-11-14 16:08:26,2017-11-14 16:08:26,1,114.74,114.74,1,0.033333,30.0,180.0,20653.2,Medium Value
1,000161a058600d5901f007fab4c27140,2017-07-16 09:40:32,2017-07-16 09:40:32,1,67.41,67.41,1,0.033333,30.0,180.0,12133.8,Low Value
2,0001fd6190edaaf884bcaf3d49edf079,2017-02-28 11:06:43,2017-02-28 11:06:43,1,195.42,195.42,1,0.033333,30.0,180.0,35175.6,Medium Value
3,0002414f95344307404f0ace7a26f1d5,2017-08-16 13:09:20,2017-08-16 13:09:20,1,179.35,179.35,1,0.033333,30.0,180.0,32283.0,Medium Value
4,000379cdec625522490c315e70c7a9fb,2018-04-02 13:42:17,2018-04-02 13:42:17,1,107.01,107.01,1,0.033333,30.0,180.0,19261.8,Medium Value
...,...,...,...,...,...,...,...,...,...,...,...,...
99436,fffecc9f79fd8c764f843e9951b11341,2018-03-29 16:59:26,2018-03-29 16:59:26,1,81.36,81.36,1,0.033333,30.0,180.0,14644.8,Low Value
99437,fffeda5b6d849fbd39689bb92087f431,2018-05-22 13:36:02,2018-05-22 13:36:02,1,63.13,63.13,1,0.033333,30.0,180.0,11363.4,Low Value
99438,ffff42319e9b2d713724ae527742af25,2018-06-13 16:57:05,2018-06-13 16:57:05,1,214.13,214.13,1,0.033333,30.0,180.0,38543.4,High Value
99439,ffffa3172527f765de70084a7e53aae8,2017-09-02 11:53:32,2017-09-02 11:53:32,1,45.50,45.50,1,0.033333,30.0,180.0,8190.0,Low Value


In [40]:
segment_recommendation_engine(rfm, segment_summary)

,segment,customer_count,avg_recency,avg_frequency,avg_monetary,priority_score,campaign_type,channel,timing
0,Champions,1623,108.08,1.0,409.26,10,VIP Retention,Email + Phone,Immediate
1,Loyal Customers,23346,299.76,1.0,248.39,8,Loyalty Program,Email,Within 14 days
2,Potential Loyalists,24590,162.77,1.0,111.65,7,Engagement,Email + Push,Within 14 days
3,New Customers,6348,108.30,1.0,167.61,6,Onboarding,Email,Immediate
4,Regular,3021,217.94,1.0,255.60,5,General,Email,Monthly
5,Price Sensitive,15722,379.57,1.0,62.43,5,General,Email,Monthly
6,Hibernating,24791,417.02,1.0,160.65,5,General,Email,Monthly


In [49]:
satisfaction_summary, review_insights, top_narratives = customer_satisfaction_analysis(order_reviews_df)
satisfaction_summary

,review_score,total_reviews,reviews_with_title,reviews_with_message,percentage,satisfaction_level,comment_rate
0,1,11424,1873,8745,11.51,Very Dissatisfied,76.55
1,2,3151,478,2145,3.18,Dissatisfied,68.07
2,3,8179,824,3557,8.24,Neutral,43.49
3,4,19142,1735,5976,19.29,Satisfied,31.22
4,5,57328,6658,20554,57.78,Very Satisfied,35.85


In [50]:
review_insights

{'total_reviews': 99224,
 'avg_score': np.float64(4.08642062404257),
 'satisfaction_rate': np.float64(77.06804805289042),
 'dissatisfaction_rate': np.float64(14.688986535515603),
 'reviews_with_comments': np.int64(40977),
 'comment_rate': np.float64(41.29746835443038),
 'median_score': np.float64(5.0),
 'score_std': np.float64(1.3475791311150982),
 'nps_score': np.float64(62.379061517374836),
 'engaged_customers': 42559}

In [43]:
top_narratives

{'positive_themes': [{'theme': 'product',
   'mentions': 10663,
   'percentage': 40.19},
  {'theme': 'deadline', 'mentions': 7024, 'percentage': 26.48},
  {'theme': 'delivery', 'mentions': 4856, 'percentage': 18.3},
  {'theme': 'excellent', 'mentions': 4185, 'percentage': 15.77},
  {'theme': 'good', 'mentions': 3941, 'percentage': 14.85},
  {'theme': 'arrived', 'mentions': 3928, 'percentage': 14.81},
  {'theme': 'recommend', 'mentions': 3870, 'percentage': 14.59}],
 'negative_themes': [{'theme': 'not', 'mentions': 6496, 'percentage': 59.65},
  {'theme': 'product', 'mentions': 5785, 'percentage': 53.12},
  {'theme': 'delivery', 'mentions': 1763, 'percentage': 16.19},
  {'theme': 'arrived', 'mentions': 1112, 'percentage': 10.21},
  {'theme': 'deadline', 'mentions': 876, 'percentage': 8.04},
  {'theme': 'return', 'mentions': 451, 'percentage': 4.14},
  {'theme': 'quality', 'mentions': 421, 'percentage': 3.87}],
 'improvement_areas': [{'area': 'return',
   'frequency': 451,
   'priority': 

In [52]:
review_sentiment_trends(order_reviews_df, orders_df)

c:\Users\kenia\OneDrive\Documents\GitHub\cobre\scripts\key_metrics.py:912: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  lambda x: (x['review_score'] >= 4).mean() * 100
c:\Users\kenia\OneDrive\Documents\GitHub\cobre\scripts\key_metrics.py:917: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  lambda x: ((x['review_score'] >= 4).sum() - (x['review_score'] <= 2).sum()) / len(x) * 100


,review_month,avg_score,total_reviews,score_std,reviews_with_comments,satisfaction_rate,nps
0,2016-09,1.00,4,0.00,4,0.00,-100.0
1,2016-10,3.57,321,1.69,234,64.17,35.0
2,2016-12,5.00,1,NaN,1,100.00,100.0
3,2017-01,4.07,797,1.37,372,76.16,61.0
4,2017-02,4.02,1776,1.39,807,74.32,58.0
5,2017-03,4.07,2676,1.34,1147,76.64,62.0
6,2017-04,4.04,2394,1.35,1008,75.65,60.0
7,2017-05,4.14,3703,1.28,1527,78.45,66.0
8,2017-06,4.15,3250,1.29,1366,78.58,66.0
9,2017-07,4.17,4032,1.26,1720,79.04,67.0
